# Who owns Lausanne? 

## 1. Public data and owners

We obtained ftp access from the Lausanne office of cadastre. The data is a collection of ESRI shapefiles, describing roads, buildings, parcels, trees, waterbodies, and others.
Each shapefile is a collection of features, and each feature has an associated geometry (e.g. the shape of a land parcel) and associated attributes ( e.g. the commune responsible for the parcel, the parcel number).

We can explore this dataset by using GIS software that supports shapefiles. We used QGIS to explore the dataset.
We hoped to find an attribute describing the parcel owner in the parcel shapefile layer, but it wasn't there.
We had to resort to web scraping to recover this attribute.

## 1.1 Scraping owners

### 1.1.2 Download XML files
We wanted to associate each parcel in Lausanne to an owner. To do this, we divided Lausanne's surface in rectangles, and requested parcel informations for these rectangles to a service exposing the owners name.
The code for the scraping is in [`/scraping/owners/scrape_owners_to_xml.py`](/edit/scraping/owners/scrape_owners_to_xml.py).

The result of owner scraping is a set of 400 xml files, each containing parcel information for a geographical rectangle. The data are saved in the following directory: `data/raw/owners/`

For privacy reason we decided not to push any data on the online github respository.

We start exploring the raw owner xml data:

In [ ]:
!ls "data/raw/owners/" | wc -l

Each file is named after the coordinates, in the Swiss systems, of the top-left and bottom-right points bounding the scraped rectangle.

In [ ]:
!ls "data/raw/owners/" 2>/dev/null | head -10 # suppress error message by redirecting errors to null

One example file:

In [ ]:
!head -n 20 "data/raw/owners/534810.4210526316_155847.0_535161.3710526315_155589.0.xml"

### 1.1.2 From XML to a single JSON
We use the `xq` and `jq` programs to extract only the features we care about from the different XML files and save them as a list of objects in a single json file.
[`scraping/owners/multiple_xml_to_single_json.sh`](/edit/scraping/owners/multiple_xml_to_single_json.sh) is a small script leveraging the expressiveness of `jq` to efficiently concatenate the XML files into a single json, while also discarding all the attributes we have no interest in.

### 1.1.3 Remove duplicated and  clean owners JSON
The generated `proprio.json` JSON has duplicate entries, entries concerning other communes than Lausanne and entries with missing owners. Furthermore, JSON is not the best format to handle tabular data. The code in [`scraping/owners/owner_json_to_clean_csv.py`](/edit/scraping/owners/owner_json_to_clean_csv.py) cleans the duplicates and tranfsorms the data into a CSV file.

### 1.1.4 Joining the owners data with the cadastre shapefiles
The result of the previous preprocessing steps is a CSV file with three columns: commune number, parcel number, and the owner name:

In [ ]:
!head 'data/owners/proprio_lausanne.csv'

We would like to add the owner name to the attributes of the parcels shapefile that we obtained from "Office du Cadastre". To do so, we import the csv and the shapefile in QGIS, and we join this two "tables" by parcel number. The resulting geographical layer contains all the geographical features representing the parcels, and additionally the owner name for each parcel. We can now export this layer as a GeoJSON, making sure to use `WGS-84` as the coordinate system,  and continue our exploration.

## 1.2 Cadastral data - data exploration

We now have a GeoJSON, containing parcel geometries and parcel owners.

In [ ]:
#imports
import json
import pandas as pd
import matplotlib.pyplot as plt
import folium

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

In [ ]:
with open('data/owners/parcelles_wgs84.geojson') as geojson:
    geo = json.load(geojson)

# geo.features contains the list of parcelles
# example:
geo['features'][0]

We load the attributes of each geographical feature in Pandas:

In [ ]:
features = [
    {'parc_num':feature['properties']['NO_PARC'],
    'owner':feature['properties']['proprio']} for feature in geo['features']
]
parcels = pd.DataFrame.from_records(features)
parcels.head()

### 1.2.1 Who is the biggest real estate owners?

We can now quickly answer questions such as who are the 30 biggest property owners in Lausanne, by using the number of parcels owned as a measure:

In [ ]:
parcels_per_owner = parcels['owner'].value_counts()
parcels_per_owner.head(30)

We can see that all thirty of the biggest owners are either corporations, pension funds, or public institutions. There is not a single private person in the list.

### 1.2.2 Unique owners

In [ ]:
owners = parcels['owner'].dropna()
print('total parcels', len(owners))
print('unique owners', len(owners.unique()))

There are 8k parcels in Lausanne. We are interestd to know how many people/societies own them. This number doesn't account for PPE (_prorpiété par étage_, single flats owned by privates). A lower bound on the owners can be estimated by discarding the PPE entreis alltogether.

In [ ]:
len(owners[~owners.str.contains('PPE ')].unique())

This is a lower bound for the number of owners. Although, the real number is likely to be much higher since it's unlikely that most of these unique owners are also owners of a PPE share.

### 1.2.3 Visualizing the distribution of missing values

In [ ]:
# Compute portion of missing values
parcels['owner'].isna().mean()

22% of the parcels don't have owner information. Indeed, many parcels reperesent roads, and as such they didn't have an owner on the site we scraped. Also we didn't scrape the values for the northern part of Lausanne, which is mostly farmland and woods.
Let's try to visualize the missing values on a map:

In [84]:
import folium
def getMap():
    return folium.Map([46.524, 6.63], tiles='cartodbpositron', zoom_start=14)

In [85]:
map_ = getMap()

def style_function(feature):
    """Returns color red for missing values, blue for valid."""
    return {
        'fillColor':
            'red' if feature['properties']['proprio'] is None else 'blue', 
        'stroke': False
    }

geo_fol = folium.GeoJson(geo, style_function=style_function)

map_.add_child(geo_fol)
map_.save('map/missing_values.html')
map_

NameError: name 'geo' is not defined

This visualization is not very snappy or legible, but we can interpret it as follows:

- Red areas are parcels for which the proprietary is `None`. The northern parts of Lausanne were not scraped, since we didn't want to overload the scraped website and since they're mostly rural areas. It is expected that they are red.
- Zooming into central Lausanne, we see that roads have unknown owners. This is also expected.
- For some areas blue and red overlap, yielding purple parcels. This is because the dataset is slightly dirty and some bigger parcels with no owner _contain_ smaller parcels with known owners. Therefore the colors overlap.

Having asserted that the dataset is fairly sane, we can drop the features were the owner is `None`, since they will be of no use to us (roads), and will make the map drawing slower.

In [ ]:
geo_clean = geo.copy()

In [ ]:
# replace the list of features by filtering out the 
# features having None as proprio
geo_clean['features'] = [
    feature for feature in geo['features']
    if feature['properties']['proprio'] is not None
]

geo = geo_clean
features = [
    {'parc_num':feature['properties']['NO_PARC'],
    'owner':feature['properties']['proprio']} for feature in geo['features']
]
parcels = pd.DataFrame.from_records(features)

###  1.2.4 Show parcels by owner type

The parcel owner format allows us to know the category of each owner. 
We will use similar categories as the statistical office of the city of Lausanne [here](https://www.lausanne.ch/officiel/statistique/quartiers/tableaux-donnees.html):

- privates
- public institutions
- companies (corporations)
- cooperatives
- pension funds
- foundations
- PPE

Societies are detected by having 'AG' or 'SA' in their name. Similary for cooperatives, foundations, and pension funds. We display a map colored by the owner category.

In [ ]:
def categorize(owner):
    if owner is None:
        return None
    owner = owner.lower()
    
    if owner == 'commune de lausanne' or\
        'dfire' in owner or\
        'cff' in owner or \
        'public' in owner:
        return 'public'
    elif 'retraites' in owner or\
        'pensions' in owner or\
        'prévoyance' in owner:
        return 'pension'
    elif 'fondation' in owner:
        return 'foundation'
    elif ' s.a.' in owner or\
        ' sa' in owner or\
        ' ag' in owner or\
        'société anonyme' in owner:
        return 'AG'
    elif 'ppe ' in owner or\
        'copropriété' in owner:
        return 'PPE'
    elif 'société coopérative' in owner:
        return 'coop'
    else:
        # not true but let's approximate
        return 'private' 

In [ ]:
owners_categories = parcels[['owner']]
owners_categories['category'] = owners_categories['owner'].apply(categorize)
owners_categories = owners_categories.drop_duplicates().set_index('owner')

In [ ]:
map_ = getMap()

def style_function(feature):
    colors = {
        'coop': 'yellow',
        'AG' : 'red',
        'public' : 'green',
        'private': 'blue',
        'PPE': 'orange',
        'pension': 'purple',
        'foundation' : 'brown'
        
    }
    owner = feature['properties']['proprio']
    cat = owners_categories.loc[owner][0]
    
    return {
        'stroke':False,
        'fillColor': colors[cat]
    }

folium.GeoJson(
    geo_clean, 
    style_function=style_function, 
    
    # show the owner at hover
    tooltip=folium.GeoJsonTooltip(['proprio'])
).add_to(map_)

map_.save("map/owner.html")
map_

## 2. Rents data

### 2.1 Scraping

In order to analyse how ownership patterns influence prices, we needed to complement the owners dataset with rent prices.
Rent prices are generally not public, but we can scrape from real estate websites' current rent listings, and extract the prices from there.
We therefore proceeded to scrape [anibis.ch](https://www.anibis.ch/fr/default.aspx), [homegate.ch](https://www.homegate.ch/fr) and [tutti.ch](https://tutti.ch) to extract up-to-date data points.
The scripts used are the following:

- for homegate: [`scraping/homegate/scrape_homegate.py`](/edit/scraping/homegate/scrape_homegate.py), a single well structured python program

- for anibis:
    1. [`scraping/anibis/anibis_scrape_listings.py`](/edit/scraping/anibis/anibis_scrape_listings.py) to download the index of results matching rents in lausanne
    2. [`scraping/anibis/anibis_parse_listings.py`](/edit/scraping/anibis/anibis_scrape_listings.py) to parse the listings index.
    3. [`scraping/anibis/anibis_scrape_offers.py`](/edit/scraping/anibis/anibis_scrape_offers.py) to download each single rent offer, given a parsed index
    4. [`scraping/anibis/anibis_parse_offers.py`](/edit/scraping/anibis/anibis_parse_offers.py) to parse the pages for each offer.
    5. [`cleaning/parse_street_and_number.py`](/edit/cleaning/parse_street_and_number.py) parses addresses to street name and number
- for tutti:
    1. [`scraping/tutti/tutti_scrape_listings.py`](/edit/scraping/tutti/tutti_scrape_listings.py) to download the index of results matching rents in lausanne
    2. [`scraping/tutti/tutti_parse_listings.py`](/edit/scraping/tutti/tutti_scrape_listings.py) to parse the listings index. data

We agreed on a shared format to simplify the merging of these three data sources.

### 2.2 Removing duplicates

Most rent listings are published on several websites. When merging the data sources, we first need to figure out which results are present in multiple datasets to avoid duplicate datapoints. We consider listings to be duplicates if they have the same address and the same price. The code is in [`cleaning/merge_rent_offers.py`](/edit/cleaning/merge_rent_offers.py) .

In [1]:
import pandas as pd
import numpy as np
from numpy import char as npc
import requests
from bs4 import BeautifulSoup
import scipy.stats

import re
import os
import json
from datetime import datetime

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%load_ext autoreload
%autoreload 2

np.seterr(all='raise');

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

In [5]:
import cleaning as cleaning

In [6]:
filenames = ["./data/rents/tutti.json", 
             "./data/rents/anibis_with_streets.json",
            "./data/rents/homegate.json"]
merged = cleaning.merge_rent_offers.main(filenames)
#print(json.dumps(merged, ensure_ascii=False, indent=2))

In [7]:
# Save to JSON
with open('./data/rents/all_rents.json', 'w') as outfile:
        json.dump(merged, outfile)

### 2.3 Mapping street addresses to coordinates
The data cleaning up to now provided us with a list of json objects, each one representing a rent offer.

In [77]:
!head -25 data/rents/all_rents.json

[{"meuble": false, "price": "120.00", "address": "0791943014", "city": "1006 Lausanne (VD)", "postCode": "1006", "surface": "120", "numberRooms": "120.000", "street": "", "number": "0791943014"}, {"real_estate_type": "Appartements", "epoch_time": 1541178363, "id": "24418273", "address": "1 Avenue du Temple", "postCode": "1012", "price": " 1888", "title": "Appartement de 3 pi\u00e8ces au 3\u00e8me \u00e9tage", "user_alias": "Domicim Lausanne G\u00e9rance", "numberRooms": "3", "surface": "78", "annount_type": "Offres de location", "monthlyPayment": true, "street": "1 av du Temple", "number": "1 "}, {"real_estate_type": "Maisons", "epoch_time": 1538381587, "id": "22572145", "address": "1066 Epalinges", "postCode": "1066", "price": " 1590000", "title": "Jolie maison mitoyenne avec 3 studios ind\u00e9pendants", "user_alias": "ActivImmob Lausanne", "numberRooms": "5.5", "surface": "142", "annount_type": "Offres de vente", "monthlyPayment": false, "street": "1066 Epalinges", "number": "10"}, 

_estate_type": "Appartements", "epoch_time": 1542798743, "id": "22417646", "address": "Rue de l`Abeille 3b, VD", "postCode": "1815", "price": " 1500", "title": "Specieux apartement au coeur de Clarens", "user_alias": "Delta Treuhandb\u00fcro", "numberRooms": "2.5", "surface": "67", "annount_type": "Offres de location", "monthlyPayment": true, "street": "Rue de l`Abeille 3b, VD", "number": "3b"}, {"address": "Rue de la Borde", "street": "Rue de la Borde", "number": "", "postCode": "1018", "city": "Lausanne", "price": "1330", "surface": "38", "numberRooms": "2", "title": "305 - App.2.0p.,3\u00e8me \u00e9tage"}, {"real_estate_type": "Appartements", "epoch_time": 1541567629, "id": "24510179", "address": "Rue de la Borde 37", "postCode": "1018", "price": " 2070", "title": "Rue de la Borde 37", "user_alias": "R\u00e9gie Duboux SA", "numberRooms": "3", "surface": "62", "annount_type": "Offres de location", "monthlyPayment": true, "street": "Rue de la Borde 37", "number": "37"}, {"meuble": fal

The address is in textual form. Also, it is clear that there will be more cleaning needed. The first address is a phone number instead of an actual address. This sanitisation is automatically provided by the next script.

To perform geographical queries on addresses, we need to convert them to coordinates. To do so, we use the cadastral layer of building addresses, provided by the Cadastral offic of Lausanne.
During merging of the anibis and homegate datasets, the addresses were standardized to use the format used by this cadastral layer.

To map an address to a coordinates couple, we iterate over all buildings in Lausanne, and check if the street name and the street number match those of our address. If there's a match, we extract the coordinates of the building from the cadastral layer. If there isn't we drop the offer (like the phone number above) and therefore perform some cleaning:

In [78]:
offers = cleaning.address_to_coords.main('./data/rents/all_rents.json')

In [79]:
# Save it as 'rent_prices.json'

# Save to JSON
with open('./data/rents/rent_prices.json', 'w') as outfile:
        json.dump(offers, outfile)

### 2.4 Visualizing the rents dataset

Finally, we can take a first look at the rental data in a cleaned form.

In [80]:
# load the json of rent prices
rent_prices = json.load(open('data/rents/rent_prices.json'))

In [81]:
# load the geojson featuring borders for each quartier
quartiers = json.load(open('data/maps/quartiers.geojson'))

In [82]:
# compute the cost per squared meter of the rent
for offer in rent_prices:
    offer['CHF/m2'] = float(offer['price'])/float(offer['surface'])

In [86]:
# draw a map showing the location of each vacancy, and the quartiers borders
m = getMap()
folium.GeoJson(quartiers).add_to(m)
for offer in rent_prices:
    coords = offer['position']
    # Marker wants first the N coordinate and then E
    folium.Marker((coords[1], coords[0]), tooltip=offer['CHF/m2']).add_to(m)
m

We see that several points fall outside of the city borders. But we still have a significant number of avaialable datapoints. We will soon drop the outliers.

### 2.5 Mapping rent datapoints to quartiers
Each rent data-point has a pair of coordinates localizing it in space. _quartiers_ are polygons, whose perimeter is a list of coordinates. We can use the python library `shapely`, that allows us to perform geometrical queries, to find the _quartier_ for each rent offer.

In [88]:
#import the two data structures needed
from shapely.geometry import Point, Polygon

for offer in rent_prices:
    offer['quartier'] = None
    for quartier in quartiers['features']:
        
        # skip because we don't have owner data for forest areas
        if quartier['properties']['Name'] == '90 - Zones foraines':  
            continue
        
        offer_pos = Point(offer['position'])
        
        # we extract the list of coordinates of the polygon's vertices, 
        # discarding useless height
        quartier_vertices = [(east, north) for east, north, z in quartier['geometry']['coordinates'][0]]
        quartier_poly = Polygon(quartier_vertices)
        if quartier_poly.contains(offer_pos):
            offer['quartier'] = quartier['properties']['Name']

Let's sanity check by changing the color of the marker depending on the found _quartier_ and displaying all of it on a map.

In [89]:
m = getMap()
folium.GeoJson(quartiers).add_to(m)
for offer in rent_prices:
    coords = offer['position']
    
    # little hack to assign a different color to each quartier
    # calculate hex color from a hash of the name
    color = '%06x' % (hash(offer['quartier']) % (256**3))
    
    # Marker wants first the N coordinate and then E
    folium.CircleMarker(
        (coords[1], coords[0]),
        radius=5, fill_color='#'+color, weight=0, fill_opacity=1
    ).add_to(m)
m

This looks pretty good. We will now give a first statistic on rent prices. **However**, there are still fake offers (like offers for parking lots and the like) and there are still outliers in the dataset. The results are therefore not yet _real, clean means_. 

As a cheap mitigation we will display the median instead of the mean. Before analysing and building our mathematical model we will however clean those offers out.

In [90]:
#display median price per neighborhood
rents_per_quartier = pd.DataFrame.from_dict(rent_prices)
rents_per_quartier[['CHF/m2', 'quartier']].groupby('quartier').agg('median')

,CHF/m2
quartier,
01 - Centre,26.784736
02 - Maupas / Valency,26.410256
03 - Sébeillon / Malley,26.461538
04 - Montoie / Bourdonnette,29.166667
05 - Montriond / Cour,33.311052
06 - Sous-Gare / Ouchy,25.331869
07 - Montchoisi,27.786070
08 - Florimont / Chissiez,28.392857
09 - Mousquines / Bellevue,26.000000


**--> clean fake offers and outliers**

## 3. Linear model describing relation of ownership and price

The data is now in the form we need in order to apply our model. As our main goal is to understand the rent price composition, we will perform linear regression on the rent prices.

More precisely, we will try to predict the prices of rents in each quartier based on the features:

 - ownership proportion of each ownership type: ($f_{public}, f_{s.a.}, $...)
 - distance from the centre of the city: $dist$
 - the mean price of rents in the _quartier_ $q$ (dependent variable): $price(q)$ 
 
The linear model is then:

$$
price(q) = \beta_1 f_{public}(q) + \beta_2  f_{s.a.}(q) + ~...~ + \beta_j  f_{privates}(q)+  \beta_k dist(q)
$$

We will apply linear regression to this model and extract the knowledge from the parameters $\beta$. One problem could however be, that the ownership pattern itself depends on the distance or vice-versa. In that case we'll be able to check this assumption by predicting the distance from the ownership types:

$$
dist(q) = \beta_1 f_{public}(q) + \beta_2  f_{s.a.}(q) + ~...~ +\beta_j  f_{privates}(q)
$$

## What comes next:

Please find more information about further ideas, our current state of work and our story in the `README`.

Because obtaining the data from all the different sources and with various methods is a huge amount of work in this project, we are not fully done with cleaning and analysing all of the datasets. As said before, the rental data still contains unwanted entries. And the data from tutti.ch has not yet been converted to the correct JSON format.

However, while working on this and discussing the progress of the project we came up with a good guideline: understand how prices are determined. This is good because it lends itself well for writing a story but it also naturally yields the rather simple mathematical model from above. We are therefore convinced that we will be able to carry out the analysis to its full extent and to come up with a good web data story in the end!

### Todos and future sections:

 - Adapt ownership categories
 - Clean fake offers and outliers
 - Linear regression on data and analysis
     - Tune model
     - extract parameters and CIs
     - intra-quartier effects
     - ...
 - Political analysis by hand
 - Check: do we answer the four RQs?
 - Graphics and maps production for web story
 
--> The outline for the story (our end result) can be found in the `README`.
 
 - Condense information to one line about finding affordable accommodation and why it is difficult (e.g. cheap parts are far from centre/university...)
 - Write the story
 - Design web page and animations (using a static generator and `JS`)
 - Deploy site to server (github pages or own)
 
**Milestone 3, 16.12.18**
 
 - Boil analysis down to 5 keypoints
 - Think about way of presenting this highly geographical data/problem
 - Write text for presentation and exercise!
